## 641 - NLP
## Final Project

### Prepared By: Ricardo Zambrano
### NN Models with GloVe Embedings

In [3]:
import os
import sys
import copy
import codecs
import gzip

import gensim
import smart_open
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

import time
import datetime
from datetime import date
from datetime import datetime
from tqdm import tqdm

import collections
from typing import NamedTuple
import random
import json
import re
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn import model_selection
from sklearn import naive_bayes
from sklearn import svm

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [4]:
import spacy
from nltk.tokenize import word_tokenize

In [5]:
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Activation, Dropout, Dense, Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer

**Loading Labeled Data and Stop Words**

In [6]:
# Setup the path where labeled text files are located 
txtFilesPath = r"C:\Users\rzamb\Desktop\Desktop\UMD\641_Natural_Language_Processing\finalProject\txtFiles"

In [7]:
def extractLalebedFile(filePath):
    """Assumes a file path for a labeled news article saved as a .txt file, a string.
    Returns a dictionary with the news article data"""
    
    # Define regular expression to recognize labels in the labeled text files
    re1 = re.compile(r"^source")
    re2 = re.compile(r"^date")
    re3 = re.compile(r"^section")
    re4 = re.compile(r"^byline")
    re5 = re.compile(r"^inflationPosition")
    re6 = re.compile(r"^title")
    re7 = re.compile(r"^subtitle")
    re8 = re.compile(r"body")
    re9 = re.compile(r"end")
    
    # Open the file in the given path and read the lines in the .txt file
    with open(filePath,'r',encoding="utf8") as f:
        lines = f.readlines()

        textLines = [] # Initializes a list to store the lines of the body of the news article

        for indx in range(len(lines)):
            if bool(re1.search(lines[indx])):
                source = re.split(r":",lines[indx])[1].strip().replace('"', '') # Extracts the source of the article
            if bool(re2.search(lines[indx])):
                dateTxt = re.split(r":",lines[indx])[1].strip().replace('"', '') # Extracts the date of the publication...
                try:
                    datePublish = datetime.strptime(dateTxt, '%Y-%m-%d')             # ... and saves it as a datetime
                except:
                    print("Cannot cast date at ",filePath," article as datetime")
                    datePublish = np.nan
            if bool(re3.search(lines[indx])):
                paperSection = re.split(r":",lines[indx])[1].strip().replace('"', '') # Extracts the section of the publication
            if bool(re4.search(lines[indx])):
                byline = re.split(r":",lines[indx])[1].strip().replace('"', '') # Extracts the author name(s)
            if bool(re5.search(lines[indx])):
                inflationLabel = re.split(r":",lines[indx])[1].strip().replace('"', '') # Extracts the label given to the article
            if bool(re6.search(lines[indx])):
                currTitle = re.split(r":",lines[indx])[1].strip().replace('"', '') # Extracts the title of the news piece
            if bool(re7.search(lines[indx])):
                currSubtitle = re.split(r":",lines[indx])[1].strip().replace('"', '') # Extracts the subtitle of the news piece
            if bool(re8.search(lines[indx])):
                bodyStart = indx+1 # Recoords the line number where the body of the article starts
            if bool(re9.search(lines[indx])):
                bodyEnd = indx # Recoords the line number where the body of the article ends
                
        for indx in range(bodyStart,bodyEnd): # Extracts and format the main text of the article
            currLine = lines[indx].lower() # .lower() replaces capital letters with lower case letters
            currLine = currLine.translate(str.maketrans('', '', string.punctuation)) # To remove punctuation
            currLine = currLine.replace("\n", "") # To remove new line \n

            textLines.append(currLine)
            
        #articleInfo = (source,datePublish,paperSection,currTitle,currSubtitle,byline,inflationLabel)
        mainText = ' '.join(textLines) # Joins the lines of the article in a single string

        # Final touches to pre-process the main text of the article
        # To eliminate trailing or leading space

        if mainText[0].isspace():
            mainText = mainText[1:]

        if mainText[len(mainText)-1].isspace():
            mainText = mainText[:(len(mainText)-1)]
        
        #Builds the dictionary with the news article and the metadata associated with the article
        articleData = {'authors':byline,
                       'date_publish':datePublish,
                       'section':paperSection,
                       'publisher':source,
                       'title':currTitle,
                       'subtitle':currSubtitle,
                       'label':inflationLabel,
                       'maintext':mainText
                      }

        f.close()
        
        return(articleData)

In [8]:
def readLabeledFiles(directory):
    """Assumes a path to a directory where labeled news articles are saved in .txt format, a strig
    Returns a dictionary with all article data, the article data is a dict"""
    
    txtFilesPath =  directory
    listTxtFiles = os.listdir(txtFilesPath)
    
    labeledArticles = {}
    
    for indx in range(len(listTxtFiles)):
        currFilePath = os.path.join(txtFilesPath,listTxtFiles[indx]) # Creates the path for a specific file
        articleMetadata = extractLalebedFile(currFilePath) # Extracts the data from the labeled news article
        labeledArticles[indx] = articleMetadata # Adds article to dict
    
    return(labeledArticles)

In [9]:
# Read a set of stoplist words from filename, assuming it contains one word per line
# Return a python Set data structure (https://www.w3schools.com/python/python_sets.asp)
def load_stopwords(filename):
    stopwords = []
    with codecs.open(filename, 'r', encoding='ascii', errors='ignore') as fp:
        stopwords = fp.read().split('\n')
    return set(stopwords)

# Loading stopwords in order to get smaller vectors
stopwords_file = 'mallet_en_stoplist.txt'
stop_words = load_stopwords(stopwords_file)
stop_words = list(stop_words)

In [10]:
labeledNews = readLabeledFiles(txtFilesPath)

Cannot cast date at  C:\Users\rzamb\Desktop\Desktop\UMD\641_Natural_Language_Processing\finalProject\txtFiles\file18.txt  article as datetime


In [11]:
# Check that the dict was loaded correctly
labeledNews[0]

{'authors': 'Ana Swanson',
 'date_publish': datetime.datetime(2022, 8, 25, 0, 0),
 'section': 'Section B; Column 0; Business/Financial Desk; Pg. 2',
 'publisher': 'The New York Times',
 'title': 'Consumer Demand Is Key To Rise in U.S.',
 'subtitle': '',
 'label': 'Expect Inflation',
 'maintext': 'research has found that americans spending during the pandemic accounted for about 60 of inflation from 201921  supply chain bottlenecks and labor shortages have been a major factor driving inflation in the united states though surging consumer demand ultimately did more to drive up prices in the last two years according to researchers at the federal reserve bank of new york the university of maryland and harvard university  in a blog post on wednesday julian di giovanni the head of climate risk studies in the new york feds research and statistics group summarized findings from a paper presented in june that found higher consumer demand for all types of products during the pandemic was respons

In [12]:
# I am creating this class in order to keep the labeled articls in an inmutable data structure
class labeledArticle(NamedTuple):
    """A class for news articles that have been labeled by a human"""
    articleID: int
    label: str
    body: str

In [13]:
# Creating instances of labeledArticle. The idea is to be able to shuffle and split the articles into 
# training set and test set while keeping track of the labels and article IDs
articlesSample = []
for key,value in labeledNews.items():
    articlesSample.append(labeledArticle(articleID=key,label=value.get('label'),body=value.get('maintext')))

In [14]:
articlesSample[:2]

[labeledArticle(articleID=0, label='Expect Inflation', body='research has found that americans spending during the pandemic accounted for about 60 of inflation from 201921  supply chain bottlenecks and labor shortages have been a major factor driving inflation in the united states though surging consumer demand ultimately did more to drive up prices in the last two years according to researchers at the federal reserve bank of new york the university of maryland and harvard university  in a blog post on wednesday julian di giovanni the head of climate risk studies in the new york feds research and statistics group summarized findings from a paper presented in june that found higher consumer demand for all types of products during the pandemic was responsible for roughly 60 percent of the inflation in the united states between 2019 and 2021  supply shocks  which include shortages of workers raw materials and shipping containers needed to produce and move goods globally  accounted for the

In [15]:
print('label: ',articlesSample[0][1])
print('body: ',articlesSample[0][2])

label:  Expect Inflation
body:  research has found that americans spending during the pandemic accounted for about 60 of inflation from 201921  supply chain bottlenecks and labor shortages have been a major factor driving inflation in the united states though surging consumer demand ultimately did more to drive up prices in the last two years according to researchers at the federal reserve bank of new york the university of maryland and harvard university  in a blog post on wednesday julian di giovanni the head of climate risk studies in the new york feds research and statistics group summarized findings from a paper presented in june that found higher consumer demand for all types of products during the pandemic was responsible for roughly 60 percent of the inflation in the united states between 2019 and 2021  supply shocks  which include shortages of workers raw materials and shipping containers needed to produce and move goods globally  accounted for the remaining 40 percent of infl

In [16]:
# Example of list of labels: [articlesSample[x][1] for x in range(len(articlesSample))]
# Example of list of article bodies: [articlesSample[x][2] for x in range(len(articlesSample))]

**Loading word2vec pre-trained embedings**

In [17]:
filename = 'GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [18]:
print(model)

KeyedVectors<vector_size=300, 3000000 keys>


In [19]:
sims = model.most_similar('inflation', topn=10)
print(sims)

[('Inflation', 0.8240750432014465), ('inflationary_pressures', 0.8009414076805115), ('inflationary', 0.7951276302337646), ('inflationary_pressure', 0.7671657800674438), ('inflation_pressures', 0.7616896629333496), ('inflationary_expectations', 0.733973503112793), ('CPI', 0.6960082650184631), ('deflation', 0.6912639737129211), ('disinflation', 0.6745432615280151), ('interest_rates', 0.6639971733093262)]


In [20]:
vector = model['computer']  # Get numpy vector of a word
vector

array([ 1.07421875e-01, -2.01171875e-01,  1.23046875e-01,  2.11914062e-01,
       -9.13085938e-02,  2.16796875e-01, -1.31835938e-01,  8.30078125e-02,
        2.02148438e-01,  4.78515625e-02,  3.66210938e-02, -2.45361328e-02,
        2.39257812e-02, -1.60156250e-01, -2.61230469e-02,  9.71679688e-02,
       -6.34765625e-02,  1.84570312e-01,  1.70898438e-01, -1.63085938e-01,
       -1.09375000e-01,  1.49414062e-01, -4.65393066e-04,  9.61914062e-02,
        1.68945312e-01,  2.60925293e-03,  8.93554688e-02,  6.49414062e-02,
        3.56445312e-02, -6.93359375e-02, -1.46484375e-01, -1.21093750e-01,
       -2.27539062e-01,  2.45361328e-02, -1.24511719e-01, -3.18359375e-01,
       -2.20703125e-01,  1.30859375e-01,  3.66210938e-02, -3.63769531e-02,
       -1.13281250e-01,  1.95312500e-01,  9.76562500e-02,  1.26953125e-01,
        6.59179688e-02,  6.93359375e-02,  1.02539062e-02,  1.75781250e-01,
       -1.68945312e-01,  1.21307373e-03, -2.98828125e-01, -1.15234375e-01,
        5.66406250e-02, -

In [21]:
len(vector)

300

**Loading GloVe pre-trained embedings**

In [22]:
glove_input_file = '.\glove.6B\glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  This is separate from the ipykernel package so we can avoid doing imports until


(400000, 100)

In [23]:
# load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [24]:
print(model)

KeyedVectors<vector_size=100, 400000 keys>


In [25]:
sims = model.most_similar('inflation', topn=10)
print(sims)

[('rate', 0.7825247645378113), ('rates', 0.7814398407936096), ('unemployment', 0.7501060366630554), ('inflationary', 0.7441107630729675), ('growth', 0.7435855269432068), ('deflation', 0.7362003326416016), ('rising', 0.732311487197876), ('rise', 0.7258995771408081), ('slowing', 0.715856671333313), ('prices', 0.7068049311637878)]


In [26]:
vector = model['computer']  # Get numpy vector of a word
vector

array([-1.6298e-01,  3.0141e-01,  5.7978e-01,  6.6548e-02,  4.5835e-01,
       -1.5329e-01,  4.3258e-01, -8.9215e-01,  5.7747e-01,  3.6375e-01,
        5.6524e-01, -5.6281e-01,  3.5659e-01, -3.6096e-01, -9.9662e-02,
        5.2753e-01,  3.8839e-01,  9.6185e-01,  1.8841e-01,  3.0741e-01,
       -8.7842e-01, -3.2442e-01,  1.1202e+00,  7.5126e-02,  4.2661e-01,
       -6.0651e-01, -1.3893e-01,  4.7862e-02, -4.5158e-01,  9.3723e-02,
        1.7463e-01,  1.0962e+00, -1.0044e+00,  6.3889e-02,  3.8002e-01,
        2.1109e-01, -6.6247e-01, -4.0736e-01,  8.9442e-01, -6.0974e-01,
       -1.8577e-01, -1.9913e-01, -6.9226e-01, -3.1806e-01, -7.8565e-01,
        2.3831e-01,  1.2992e-01,  8.7721e-02,  4.3205e-01, -2.2662e-01,
        3.1549e-01, -3.1748e-01, -2.4632e-03,  1.6615e-01,  4.2358e-01,
       -1.8087e+00, -3.6699e-01,  2.3949e-01,  2.5458e+00,  3.6111e-01,
        3.9486e-02,  4.8607e-01, -3.6974e-01,  5.7282e-02, -4.9317e-01,
        2.2765e-01,  7.9966e-01,  2.1428e-01,  6.9811e-01,  1.12

In [27]:
len(vector)

100

**Preprocessing data**

In [28]:
X_train, X_test, y_train, y_test = train_test_split([articlesSample[x][2] for x in range(len(articlesSample))], 
                                                    [articlesSample[x][1] for x in range(len(articlesSample))], 
                                                    test_size=0.2)

In [29]:
# This function encodes the categorical labels
Encoder = LabelEncoder()
y_train_encoded = Encoder.fit_transform(y_train)
y_test_encoded = Encoder.fit_transform(y_test)

In [30]:
# Convert integers to dummy variables. Keras expects "one hot" encoding for labels in multiclass problems
y_train_labels = to_categorical(y_train_encoded)
y_test_labels = to_categorical(y_test_encoded)

In [31]:
def prepareArticles(article,stopWords):
    punct = string.punctuation+'“'+'”'+'’'
    rawTokens = word_tokenize(article)
    tokens = [token for token in rawTokens if token not in punct]
    return [word for word in tokens if not word in stopWords]

In [32]:
X_train_clean = [prepareArticles(item,stop_words) for item in X_train]
X_test_clean = [prepareArticles(item,stop_words) for item in X_test]

The next step tokenize all the words in the training set. The Tokenizer basically indexes all the words in the text and each word gets a unique index. This helps in creating dictionary mapping words to an index.

In [33]:
tokenizer = Tokenizer(num_words=5000)

In [34]:
X_train_prepros = [' '.join(item) for item in X_train_clean]
X_test_prepros = [' '.join(item) for item in X_test_clean]

In [35]:
tokenizer.fit_on_texts(X_train_prepros)

In [36]:
words_to_index = tokenizer.word_index

In [37]:
words_to_index

{'inflation': 1,
 'fed': 2,
 'prices': 3,
 'rates': 4,
 'year': 5,
 'central': 6,
 'higher': 7,
 'price': 8,
 'rate': 9,
 'economy': 10,
 'interest': 11,
 '2': 12,
 'policy': 13,
 'america': 14,
 'growth': 15,
 'federal': 16,
 'bank': 17,
 'supply': 18,
 'years': 19,
 'expectations': 20,
 'investors': 21,
 'market': 22,
 'month': 23,
 'target': 24,
 'markets': 25,
 'high': 26,
 'energy': 27,
 'low': 28,
 'mr': 29,
 'demand': 30,
 'spending': 31,
 'reserve': 32,
 'labor': 33,
 'economists': 34,
 'economic': 35,
 'world': 36,
 'pandemic': 37,
 'workers': 38,
 'banks': 39,
 'officials': 40,
 'consumer': 41,
 'time': 42,
 'index': 43,
 'months': 44,
 'government': 45,
 'percent': 46,
 'bond': 47,
 'recent': 48,
 'wages': 49,
 'powell': 50,
 'rising': 51,
 'past': 52,
 'people': 53,
 'increase': 54,
 'consumers': 55,
 'wage': 56,
 'jobs': 57,
 'rose': 58,
 'raise': 59,
 'goods': 60,
 'increases': 61,
 'money': 62,
 'long': 63,
 'fiscal': 64,
 'monetary': 65,
 'unemployment': 66,
 'big': 67,

In [38]:
len(words_to_index)

4692

In [39]:
# This snippet of code creates a dictionary that maps the words to their respective word embeddings.
word_to_vec_map = {}
wordsNotInGlove = 0
for word in words_to_index.keys():
    try:
        word_to_vec_map[word] = model[word]
    except:
        wordsNotInGlove = wordsNotInGlove + 1

wordsNotInGlove

461

**Creating an embeding matrix**
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [40]:
# Checking some metrics on the training set to define input maximum length 
print(np.mean([len(item) for item in X_train_prepros]))
print(max([len(item) for item in X_train_prepros]))
print(np.median([len(item) for item in X_train_prepros]))

3283.925
14756
3114.5


In [41]:
maxLen = 300 # originally 150 ### 5000 was too large, trying between 150 300 and 500 ###

In [42]:
vocab_len = len(words_to_index)+1#-wordsNotInGlove
embed_vector_len = 100

In [43]:
# Initialize the embeding matrix
emb_matrix = np.zeros((vocab_len, embed_vector_len))

In [44]:
# Filling the embeding matrix. Words which are not in the GloVe dictionary being assigned a zero vector.
for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        emb_matrix[index] = embedding_vector
        #emb_matrix[index, :] = embedding_vector

In [45]:
# An Embedding layer should be fed sequences of integers, i.e. a 2D input of shape (samples, indices). 
# These input sequences should be padded so that they all have the same length in a batch of input data 
# (although an Embedding layer is capable of processing sequence of heterogenous length, if you don't 
# pass an explicit input_length argument to the layer).

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, 
                            input_length=maxLen, weights = [emb_matrix], trainable=True)

**Setting up LSTM RNN and GRU RNN for text classification**

This function defines our model architecture, first, we use the embedding layer to map the words to their GloVe vectors, and then those vectors are input to the LSTM layers followed by a Dense layer with ‘sigmoid’ activation.

In [46]:
#from keras.layers import GRU
#GRU(max_len, activation='relu')

In [47]:
def lstmArchitecture(input_shape):

    X_indices = Input(input_shape)

    embeddings = embedding_layer(X_indices) 

    lstmLayer = LSTM(128, return_sequences=False,activation='relu')(embeddings)

    dropoutLayer = Dropout(0.1)(lstmLayer) # Experiment with 0.2. Originally I used 0.6 too much few data
    
    dense_1 = Dense(128, activation='relu')(dropoutLayer)

    dense_2 = Dense(3, activation='softmax')(dense_1)

    model = Model(inputs=X_indices, outputs=dense_2) 

    return model

In [48]:
def gruArchitecture(input_shape):

    X_indices = Input(input_shape)

    embeddings = embedding_layer(X_indices) 

    gruLayer = GRU(128,activation='relu')(embeddings) # Removed return_sequences=False kwarg

    dropoutLayer = Dropout(0.1)(gruLayer) # Experiment with 0.2. Originally I used 0.6 too much few data
    
    dense_1 = Dense(128, activation='relu')(dropoutLayer)

    dense_2 = Dense(3, activation='softmax')(dense_1)

    model = Model(inputs=X_indices, outputs=dense_2) 

    return model

We will convert the reviews in the dataset to their index form by using the texts_to_sequences function available with tokenizer. After that, we will pad the sequences so all of them have the same length.
Source: https://towardsdatascience.com/sentiment-analysis-using-lstm-and-glove-embeddings-99223a87fe8e

In [49]:
X_train_indices = tokenizer.texts_to_sequences(X_train_prepros)

In [50]:
X_train_indices[0]

[16,
 32,
 206,
 173,
 50,
 2055,
 1354,
 365,
 2056,
 2057,
 51,
 3,
 33,
 95,
 238,
 174,
 41,
 1,
 58,
 623,
 97,
 624,
 52,
 5,
 239,
 993,
 34,
 770,
 25,
 2058,
 81,
 311,
 51,
 98,
 432,
 99,
 2059,
 74,
 47,
 1355,
 2060,
 207,
 2061,
 52,
 19,
 99,
 150,
 74,
 366,
 516,
 517,
 129,
 62,
 189,
 2062,
 35,
 15,
 2063,
 239,
 208,
 2,
 2064,
 1356,
 98,
 994,
 11,
 4,
 28,
 63,
 10,
 1357,
 2065,
 112,
 275,
 2,
 312,
 1358,
 367,
 8,
 2066,
 625,
 82,
 2067,
 626,
 2068,
 2069,
 97,
 121,
 995,
 130,
 3,
 1359,
 83,
 27,
 2070,
 51,
 518,
 23,
 2071,
 52,
 151,
 44,
 8,
 276,
 58,
 627,
 1360,
 97,
 313,
 1361,
 54,
 2072,
 2073,
 519,
 771,
 56,
 240,
 8,
 314,
 48,
 44,
 2074,
 2,
 2075,
 628,
 629,
 2076,
 2077,
 65,
 996,
 2078,
 2079,
 2080,
 1362,
 2081,
 433,
 209,
 2082,
 368,
 628,
 1363,
 1364,
 1365,
 997,
 630,
 275,
 241,
 1366,
 175,
 2,
 772,
 2083,
 210,
 3,
 520,
 516,
 12,
 1,
 24,
 16,
 369,
 22,
 315,
 998,
 242,
 52,
 12,
 316,
 2084,
 277,
 35,
 15,
 13,
 

In [51]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [52]:
X_train_indices

array([[  51,    3,   33, ..., 1371,   29,   50],
       [  14,  440, 2163, ..., 2221, 2222,  531],
       [2223,    7,    1, ...,    0,    0,    0],
       ...,
       [4462, 4463,  110, ...,   79,  622,    3],
       [  52,  122,  118, ...,  270, 1541, 4603],
       [4615,  992, 1281, ...,    1, 4692,   53]])

In [53]:
X_train_indices.shape

(40, 300)

**Training the LSTM model**

In [54]:
lstmGloVe_model = lstmArchitecture((maxLen,))

In [55]:
lstmGloVe_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 100)          469300    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 603,447
Trainable params: 603,447
Non-trainable params: 0
_________________________________________________________________


In [56]:
adam = keras.optimizers.Adam(lr = 0.0001) # 0.0001, 0.0002, 0.00001
lstmGloVe_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
lstmGloVe_model.fit(X_train_indices, y_train_labels, batch_size=4, epochs=100)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
40/40 [==============================] - 4s 98ms/step - loss: 1.0708 - acc: 0.4750
Epoch 2/100
40/40 [==============================] - 3s 68ms/step - loss: 1.0538 - acc: 0.5500
Epoch 3/100
40/40 [==============================] - 4s 88ms/step - loss: 1.0292 - acc: 0.5750
Epoch 4/100
40/40 [==============================] - 3s 86ms/step - loss: 1.0190 - acc: 0.5500
Epoch 5/100
40/40 [==============================] - 4s 93ms/step - loss: 1.0160 - acc: 0.5500
Epoch 6/100
40/40 [==============================] - 4s 100ms/step - loss: 1.0072 - acc: 0.5750
Epoch 7/100
40/40 [==============================] - 4s 97ms/step - loss: 0.9925 - acc: 0.5500
Epoch 8/100
40/40 [==============================] - 4s 91ms/step - loss: 0.9741 - acc: 0.5500
Epoch 9/100
40/40 [==============================] - 4s 91ms/step - loss: 0.9606 - acc: 0.5750
Epoch 10/100
40/40 [==============================] - 4s 94ms/step - loss: 0.9596 - acc: 0.5750
Epoch 11/100
40/40 [============================

40/40 [==============================] - 5s 116ms/step - loss: 9.2679 - acc: 0.4250
Epoch 86/100
40/40 [==============================] - 5s 116ms/step - loss: 10.0738 - acc: 0.3750
Epoch 87/100
40/40 [==============================] - 4s 109ms/step - loss: 10.0738 - acc: 0.3750
Epoch 88/100
40/40 [==============================] - 4s 97ms/step - loss: 9.6709 - acc: 0.4000
Epoch 89/100
40/40 [==============================] - 4s 89ms/step - loss: 10.0738 - acc: 0.3750
Epoch 90/100
40/40 [==============================] - 4s 92ms/step - loss: 8.4620 - acc: 0.4750
Epoch 91/100
40/40 [==============================] - 4s 92ms/step - loss: 9.6709 - acc: 0.4000
Epoch 92/100
40/40 [==============================] - 4s 88ms/step - loss: 7.6561 - acc: 0.5250
Epoch 93/100
40/40 [==============================] - 4s 88ms/step - loss: 8.4620 - acc: 0.4750
Epoch 94/100
40/40 [==============================] - 4s 88ms/step - loss: 10.0738 - acc: 0.3750
Epoch 95/100
40/40 [==========================

In [57]:
X_test_indices = tokenizer.texts_to_sequences(X_test_prepros)
X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [58]:
lstmGloVe_model.evaluate(X_test_indices, y_test_labels)

10/10 [==============================] - 0s 18ms/step


[3.4285683631896973, 0.699999988079071]

In [59]:
preds = lstmGloVe_model.predict(X_test_indices)

In [60]:
print('predicted: ',preds)
print('actual: ',y_test_labels)

predicted:  [[0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.4017076  0.27765962 0.32063282]
 [0.         1.         0.        ]
 [1.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.4017076  0.27765965 0.32063282]
 [0.         1.         0.        ]
 [0.         1.         0.        ]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [63]:
print('predicted: ',np.argmax(preds,axis=1))
print('actual: ',np.argmax(y_test_labels,axis=1))

predicted:  [1 1 0 1 0 1 1 0 1 1]
actual:  [1 0 0 0 0 1 1 2 1 1]


In [64]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_1 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(preds,axis=1),target_names=target_names)
print(clsf_rep_1)

                        precision    recall  f1-score   support

      Expect inflation       0.67      0.50      0.57         4
Inflation will go away       0.71      1.00      0.83         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.70        10
             macro avg       0.46      0.50      0.47        10
          weighted avg       0.62      0.70      0.65        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [65]:
n = np.random.randint(0,10)
print(n)
print(X_test[n])
print(y_test[n])
print(y_test_encoded[n])
print('acual: ',y_test_labels[n])
print('predicted: ',preds[n])

5
we still say the course until the job is done” said jerome powell the federal reserve’s chairman on december 14th shortly after the central bank’s latest interestrate rise as a statement of intent his words were both straightforward and utterly sensible but what it means for the job to be done is becoming a matter of controversy inflation remains uncomfortably high meanwhile the aggressive monetary tightening of the past year is only now filtering through to the economy complicating assessments of whether the fed has in fact done enough to rein in prices  promisingly after a difficult two years inflation does appear to be easing its grip on the american economy overall prices increased by a mere 01 monthonmonth in november according to data published on december 13th making for that rarest of recent occurrences a downside surprise most encouraging was a breakdown showing that core inflation which strips out volatile food and energy costs had decelerated for a second consecutive month

In [66]:
sampleBody = X_test[n]
for i in range(len(articlesSample)):
    if articlesSample[i][2] == sampleBody:
        print(articlesSample[i])

labeledArticle(articleID=44, label='Inflation will fade away', body='we still say the course until the job is done” said jerome powell the federal reserve’s chairman on december 14th shortly after the central bank’s latest interestrate rise as a statement of intent his words were both straightforward and utterly sensible but what it means for the job to be done is becoming a matter of controversy inflation remains uncomfortably high meanwhile the aggressive monetary tightening of the past year is only now filtering through to the economy complicating assessments of whether the fed has in fact done enough to rein in prices  promisingly after a difficult two years inflation does appear to be easing its grip on the american economy overall prices increased by a mere 01 monthonmonth in november according to data published on december 13th making for that rarest of recent occurrences a downside surprise most encouraging was a breakdown showing that core inflation which strips out volatile f

In [67]:
#lstmGloVe_model.save_weights('\lstmGloVe_model_weights.hdf5')

**Fine tuning LSTM model**

**1- Learning Rate from 0.0001 to 0.00001**

In [68]:
maxLenV2 = 300 ## The variable name change is for reference only, since it was not changing from the original maxLen
lstmGloVe_modelV2 = lstmArchitecture((maxLenV2,))
lstmGloVe_modelV2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 100)          469300    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 603,447
Trainable params: 603,447
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Previous iteration lr = 0.0001 now lr = 0.00001
adamV2 = keras.optimizers.Adam(lr = 0.00001) # 0.0001, 0.0002, 0.00001
lstmGloVe_modelV2.compile(optimizer=adamV2, loss='categorical_crossentropy', metrics=['accuracy'])
lstmGloVe_modelV2.fit(X_train_indices, y_train_labels, batch_size=4, epochs=100)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
40/40 [==============================] - 4s 112ms/step - loss: 1.0896 - acc: 0.2500
Epoch 2/100
40/40 [==============================] - 4s 90ms/step - loss: 1.0860 - acc: 0.2750
Epoch 3/100
40/40 [==============================] - 4s 109ms/step - loss: 1.0806 - acc: 0.2250
Epoch 4/100
40/40 [==============================] - 4s 104ms/step - loss: 1.0686 - acc: 0.2500
Epoch 5/100
40/40 [==============================] - 4s 89ms/step - loss: 1.0715 - acc: 0.2500
Epoch 6/100
40/40 [==============================] - 4s 95ms/step - loss: 1.0752 - acc: 0.2750
Epoch 7/100
40/40 [==============================] - 4s 110ms/step - loss: 1.0823 - acc: 0.2250
Epoch 8/100
40/40 [==============================] - 4s 107ms/step - loss: 1.0677 - acc: 0.2500
Epoch 9/100
40/40 [==============================] - 4s 93ms/step - loss: 1.0737 - acc: 0.2750
Epoch 10/100
40/40 [==============================] - 4s 88ms/step - loss: 1.0723 - acc: 0.3000
Epoch 11/100
40/40 [========================

Epoch 86/100
40/40 [==============================] - 4s 104ms/step - loss: 0.9755 - acc: 0.7250
Epoch 87/100
40/40 [==============================] - 4s 107ms/step - loss: 0.9760 - acc: 0.7500
Epoch 88/100
40/40 [==============================] - 4s 111ms/step - loss: 0.9788 - acc: 0.6750
Epoch 89/100
40/40 [==============================] - 5s 120ms/step - loss: 0.9677 - acc: 0.7500
Epoch 90/100
40/40 [==============================] - 4s 111ms/step - loss: 0.9744 - acc: 0.7000
Epoch 91/100
40/40 [==============================] - 4s 110ms/step - loss: 0.9529 - acc: 0.7500
Epoch 92/100
40/40 [==============================] - 4s 101ms/step - loss: 0.9695 - acc: 0.6750
Epoch 93/100
40/40 [==============================] - 4s 94ms/step - loss: 0.9627 - acc: 0.7000
Epoch 94/100
40/40 [==============================] - 4s 92ms/step - loss: 0.9522 - acc: 0.7500
Epoch 95/100
40/40 [==============================] - 4s 89ms/step - loss: 0.9691 - acc: 0.7250
Epoch 96/100
40/40 [=============

In [70]:
lstmGloVe_modelV2.evaluate(X_test_indices, y_test_labels)

10/10 [==============================] - 0s 25ms/step


[1.0086145401000977, 0.6000000238418579]

In [71]:
predsV2 = lstmGloVe_modelV2.predict(X_test_indices)
print('predicted: ',predsV2)
print('actual: ',y_test_labels)

predicted:  [[0.35376057 0.32390282 0.32233655]
 [0.35376057 0.32390282 0.32233655]
 [0.35376057 0.32390282 0.32233655]
 [0.35376504 0.32390186 0.3223331 ]
 [0.4382766  0.3815464  0.18017696]
 [0.3537606  0.32390285 0.32233655]
 [0.4290003  0.37065172 0.20034799]
 [0.35376057 0.32390282 0.32233655]
 [0.3487999  0.37174937 0.27945074]
 [0.38032869 0.46068618 0.15898512]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [72]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_2 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(predsV2,axis=1), target_names=target_names)
print(clsf_rep_2)

                        precision    recall  f1-score   support

      Expect inflation       0.50      1.00      0.67         4
Inflation will go away       1.00      0.40      0.57         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.60        10
             macro avg       0.50      0.47      0.41        10
          weighted avg       0.70      0.60      0.55        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

**2- Epochs from 100 to 150**

In [73]:
maxLenV3 = 300 ## The variable name change is for reference only, since it was not changing from the original maxLen
lstmGloVe_modelV3 = lstmArchitecture((maxLenV3,))
lstmGloVe_modelV3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 100)          469300    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 603,447
Trainable params: 603,447
Non-trainable params: 0
_________________________________________________________________


In [74]:
# Original iteration epochs = 100 now epochs = 150
adamV3 = keras.optimizers.Adam(lr = 0.0001) # 0.0001, 0.0002, 0.00001
lstmGloVe_modelV3.compile(optimizer=adamV3, loss='categorical_crossentropy', metrics=['accuracy'])
lstmGloVe_modelV3.fit(X_train_indices, y_train_labels, batch_size=4, epochs=150)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/150
40/40 [==============================] - 6s 154ms/step - loss: 1.1144 - acc: 0.4000
Epoch 2/150
40/40 [==============================] - 4s 108ms/step - loss: 1.0874 - acc: 0.4750
Epoch 3/150
40/40 [==============================] - 4s 101ms/step - loss: 1.0714 - acc: 0.5500
Epoch 4/150
40/40 [==============================] - 4s 97ms/step - loss: 1.0616 - acc: 0.5250
Epoch 5/150
40/40 [==============================] - 4s 109ms/step - loss: 1.0432 - acc: 0.6000
Epoch 6/150
40/40 [==============================] - 5s 124ms/step - loss: 1.0264 - acc: 0.5500
Epoch 7/150
40/40 [==============================] - 4s 109ms/step - loss: 1.0185 - acc: 0.6750
Epoch 8/150
40/40 [==============================] - 4s 107ms/step - loss: 1.0038 - acc: 0.6750
Epoch 9/150
40/40 [==============================] - 5s 124ms/step - loss: 1.0052 - acc: 0.6250
Epoch 10/150
40/40 [==============================] - 5s 136ms/step - loss: 0.9815 - acc: 0.7500
Epoch 11/150
40/40 [====================

40/40 [==============================] - 4s 91ms/step - loss: 8.0590 - acc: 0.5000
Epoch 87/150
40/40 [==============================] - 4s 94ms/step - loss: 8.0590 - acc: 0.5000
Epoch 88/150
40/40 [==============================] - 4s 89ms/step - loss: 8.0590 - acc: 0.5000
Epoch 89/150
40/40 [==============================] - 4s 88ms/step - loss: 7.6561 - acc: 0.5250
Epoch 90/150
40/40 [==============================] - 4s 88ms/step - loss: 7.6561 - acc: 0.5250
Epoch 91/150
40/40 [==============================] - 4s 88ms/step - loss: 6.8502 - acc: 0.5750
Epoch 92/150
40/40 [==============================] - 4s 94ms/step - loss: 8.0590 - acc: 0.5000
Epoch 93/150
40/40 [==============================] - 4s 103ms/step - loss: 7.6561 - acc: 0.5250
Epoch 94/150
40/40 [==============================] - 4s 112ms/step - loss: 8.0590 - acc: 0.5000
Epoch 95/150
40/40 [==============================] - 5s 115ms/step - loss: 8.0590 - acc: 0.5000
Epoch 96/150
40/40 [==============================

In [75]:
lstmGloVe_modelV3.evaluate(X_test_indices, y_test_labels)

10/10 [==============================] - 1s 61ms/step


[9.788687705993652, 0.30000001192092896]

In [76]:
predsV3 = lstmGloVe_modelV3.predict(X_test_indices)
print('predicted: ',predsV3)
print('actual: ',y_test_labels)

predicted:  [[1.         0.         0.        ]
 [0.         1.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.4011797  0.29102105 0.30779922]
 [1.         0.         0.        ]
 [1.         0.         0.        ]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [77]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_3 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(predsV3,axis=1), target_names=target_names)
print(clsf_rep_3)

                        precision    recall  f1-score   support

      Expect inflation       0.33      0.75      0.46         4
Inflation will go away       0.00      0.00      0.00         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.30        10
             macro avg       0.11      0.25      0.15        10
          weighted avg       0.13      0.30      0.18        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

**3- Maximum sentence len MaxLen from 300 to 150 and learning rate from 0.0001 to 0.00005**

In [78]:
# With maxLen = 500 the loss function whent to infinity
maxLenV4 = 150 ## Variable change meaningful, so I have to adjust other variables that depended on maxLen

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, 
                            input_length=maxLenV4, weights = [emb_matrix], trainable=True)


X_train_indicesV4 = tokenizer.texts_to_sequences(X_train_prepros)
X_train_indicesV4 = pad_sequences(X_train_indices, maxlen=maxLenV4, padding='post')

X_test_indicesV4 = tokenizer.texts_to_sequences(X_test_prepros)
X_test_indicesV4 = pad_sequences(X_test_indices, maxlen=maxLenV4, padding='post')

lstmGloVe_modelV4 = lstmArchitecture((maxLenV4,))
lstmGloVe_modelV4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 100)          469300    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 603,447
Trainable params: 603,447
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Original lr = 0.0001 now lr = 0.00005
adamV4 = keras.optimizers.Adam(lr = 0.00005) # 0.0001, 0.0002, 0.00001
lstmGloVe_modelV4.compile(optimizer=adamV4, loss='categorical_crossentropy', metrics=['accuracy'])
lstmGloVe_modelV4.fit(X_train_indicesV4, y_train_labels, batch_size=4, epochs=100)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
40/40 [==============================] - 4s 92ms/step - loss: 1.0719 - acc: 0.5000
Epoch 2/100
40/40 [==============================] - 2s 56ms/step - loss: 1.0607 - acc: 0.5750
Epoch 3/100
40/40 [==============================] - 2s 56ms/step - loss: 1.0573 - acc: 0.5500
Epoch 4/100
40/40 [==============================] - 2s 55ms/step - loss: 1.0509 - acc: 0.5500
Epoch 5/100
40/40 [==============================] - 2s 56ms/step - loss: 1.0397 - acc: 0.6000
Epoch 6/100
40/40 [==============================] - 2s 57ms/step - loss: 1.0424 - acc: 0.5750
Epoch 7/100
40/40 [==============================] - 3s 64ms/step - loss: 1.0290 - acc: 0.6000
Epoch 8/100
40/40 [==============================] - 2s 61ms/step - loss: 1.0199 - acc: 0.6000
Epoch 9/100
40/40 [==============================] - 3s 63ms/step - loss: 1.0174 - acc: 0.5500
Epoch 10/100
40/40 [==============================] - 3s 67ms/step - loss: 1.0108 - acc: 0.5750
Epoch 11/100
40/40 [=============================

40/40 [==============================] - 2s 52ms/step - loss: 0.6481 - acc: 0.8000
Epoch 86/100
40/40 [==============================] - 2s 52ms/step - loss: 0.6636 - acc: 0.8000
Epoch 87/100
40/40 [==============================] - 2s 49ms/step - loss: 0.6553 - acc: 0.8000
Epoch 88/100
40/40 [==============================] - 2s 49ms/step - loss: 0.6323 - acc: 0.8000
Epoch 89/100
40/40 [==============================] - 2s 48ms/step - loss: 0.6616 - acc: 0.8000
Epoch 90/100
40/40 [==============================] - 2s 51ms/step - loss: 0.6323 - acc: 0.8250
Epoch 91/100
40/40 [==============================] - 2s 50ms/step - loss: 0.6258 - acc: 0.8000
Epoch 92/100
40/40 [==============================] - 2s 52ms/step - loss: 0.6059 - acc: 0.7750
Epoch 93/100
40/40 [==============================] - 2s 50ms/step - loss: 0.6223 - acc: 0.8000
Epoch 94/100
40/40 [==============================] - 2s 50ms/step - loss: 0.6095 - acc: 0.8000: 0s - loss: 0.5931 - acc: 0.805
Epoch 95/100
40/40 [=

In [80]:
lstmGloVe_modelV4.evaluate(X_test_indicesV4, y_test_labels)

10/10 [==============================] - 1s 87ms/step


[1.0156376361846924, 0.4000000059604645]

In [81]:
predsV4 = lstmGloVe_modelV4.predict(X_test_indicesV4)
print('predicted: ',predsV4)
print('actual: ',y_test_labels)

predicted:  [[0.50240266 0.21173917 0.28585815]
 [0.50240266 0.21173917 0.28585815]
 [0.50240266 0.21173917 0.28585815]
 [0.5024856  0.21166041 0.28585395]
 [0.42380327 0.44082615 0.13537061]
 [0.502403   0.21173882 0.28585818]
 [0.51898193 0.34723926 0.13377874]
 [0.50240266 0.21173917 0.28585815]
 [0.5560447  0.279491   0.16446427]
 [0.29723078 0.58076614 0.12200305]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [82]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_4 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(predsV4,axis=1), target_names=target_names)
print(clsf_rep_4)

                        precision    recall  f1-score   support

      Expect inflation       0.38      0.75      0.50         4
Inflation will go away       0.50      0.20      0.29         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.40        10
             macro avg       0.29      0.32      0.26        10
          weighted avg       0.40      0.40      0.34        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

**Training the GRU model**

In [83]:
# Adjusting variables to best variables of the LSTM model
maxLen = 150 ## 150 or 300

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, 
                            input_length=maxLen, weights = [emb_matrix], trainable=True)


X_train_indices = tokenizer.texts_to_sequences(X_train_prepros)
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

X_test_indices = tokenizer.texts_to_sequences(X_test_prepros)
X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [84]:
gruGloVe_model = gruArchitecture((maxLen,))
gruGloVe_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 100)          469300    
_________________________________________________________________
gru (GRU)                    (None, 128)               87936     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 387       
Total params: 574,135
Trainable params: 574,135
Non-trainable params: 0
_________________________________________________________________


In [85]:
# Starting with smallest lr from LSTM. If in doubt I will test 0.00005
adam = keras.optimizers.Adam(lr = 0.00001) # 0.0001, 0.0002, 0.00001
gruGloVe_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
gruGloVe_model.fit(X_train_indices, y_train_labels, batch_size=4, epochs=100)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
40/40 [==============================] - 4s 100ms/step - loss: 1.1418 - acc: 0.2250
Epoch 2/100
40/40 [==============================] - 1s 36ms/step - loss: 1.1390 - acc: 0.2250
Epoch 3/100
40/40 [==============================] - 1s 37ms/step - loss: 1.1295 - acc: 0.2750
Epoch 4/100
40/40 [==============================] - 1s 37ms/step - loss: 1.1329 - acc: 0.2250
Epoch 5/100
40/40 [==============================] - 2s 44ms/step - loss: 1.1351 - acc: 0.2000
Epoch 6/100
40/40 [==============================] - 2s 44ms/step - loss: 1.1297 - acc: 0.2500
Epoch 7/100
40/40 [==============================] - 2s 47ms/step - loss: 1.1208 - acc: 0.3500
Epoch 8/100
40/40 [==============================] - 2s 46ms/step - loss: 1.1116 - acc: 0.3500
Epoch 9/100
40/40 [==============================] - ETA: 0s - loss: 1.1191 - acc: 0.305 - 2s 49ms/step - loss: 1.1240 - acc: 0.2750
Epoch 10/100
40/40 [==============================] - 2s 48ms/step - loss: 1.1196 - acc: 0.3500
Epoch 11/1

40/40 [==============================] - 2s 44ms/step - loss: 0.9706 - acc: 0.5500
Epoch 86/100
40/40 [==============================] - 2s 46ms/step - loss: 0.9722 - acc: 0.5500
Epoch 87/100
40/40 [==============================] - 2s 44ms/step - loss: 0.9732 - acc: 0.5500
Epoch 88/100
40/40 [==============================] - 2s 40ms/step - loss: 0.9743 - acc: 0.5500
Epoch 89/100
40/40 [==============================] - 2s 41ms/step - loss: 0.9711 - acc: 0.5500
Epoch 90/100
40/40 [==============================] - 2s 41ms/step - loss: 0.9683 - acc: 0.5500
Epoch 91/100
40/40 [==============================] - 2s 46ms/step - loss: 0.9690 - acc: 0.5500
Epoch 92/100
40/40 [==============================] - 2s 48ms/step - loss: 0.9692 - acc: 0.5500
Epoch 93/100
40/40 [==============================] - 2s 48ms/step - loss: 0.9533 - acc: 0.5500
Epoch 94/100
40/40 [==============================] - 2s 48ms/step - loss: 0.9677 - acc: 0.5750
Epoch 95/100
40/40 [==============================] -

In [86]:
gruGloVe_model.evaluate(X_test_indices, y_test_labels)

10/10 [==============================] - 1s 92ms/step


[1.078713059425354, 0.4000000059604645]

In [87]:
preds2 = gruGloVe_model.predict(X_test_indices)

In [88]:
print('predicted: ',preds2)
print('actual: ',y_test_labels)

predicted:  [[0.35226646 0.32542554 0.32230797]
 [0.35226762 0.32542557 0.3223068 ]
 [0.35226646 0.32542554 0.32230797]
 [0.47480887 0.2563035  0.26888764]
 [0.44469622 0.295655   0.25964877]
 [0.4428601  0.2804908  0.2766491 ]
 [0.43489844 0.28000346 0.28509805]
 [0.35226646 0.32542554 0.32230797]
 [0.44471386 0.3254286  0.2298576 ]
 [0.42666602 0.2941859  0.27914807]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [89]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_5 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(preds2,axis=1), target_names=target_names)
print(clsf_rep_5)

                        precision    recall  f1-score   support

      Expect inflation       0.40      1.00      0.57         4
Inflation will go away       0.00      0.00      0.00         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.40        10
             macro avg       0.13      0.33      0.19        10
          weighted avg       0.16      0.40      0.23        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [90]:
n = np.random.randint(0,10)
print(n)
print(X_test[n])
print(y_test[n])
print(y_test_encoded[n])
print('acual: ',y_test_labels[n])
print('predicted: ',preds2[n])

5
we still say the course until the job is done” said jerome powell the federal reserve’s chairman on december 14th shortly after the central bank’s latest interestrate rise as a statement of intent his words were both straightforward and utterly sensible but what it means for the job to be done is becoming a matter of controversy inflation remains uncomfortably high meanwhile the aggressive monetary tightening of the past year is only now filtering through to the economy complicating assessments of whether the fed has in fact done enough to rein in prices  promisingly after a difficult two years inflation does appear to be easing its grip on the american economy overall prices increased by a mere 01 monthonmonth in november according to data published on december 13th making for that rarest of recent occurrences a downside surprise most encouraging was a breakdown showing that core inflation which strips out volatile food and energy costs had decelerated for a second consecutive month

**Setting up Bidirectional LSTM RNN and GRU RNN for text classification**

In [91]:
def biDirect_lstmArch(input_shape):

    X_indices = Input(input_shape)

    embeddings = embedding_layer(X_indices) 

    lstmLayer = Bidirectional(LSTM(128,activation='relu'))(embeddings) # return_sequences=False

    dropoutLayer = Dropout(0.1)(lstmLayer) 
    
    dense_1 = Dense(128, activation='relu')(dropoutLayer)

    dense_2 = Dense(3, activation='softmax')(dense_1)

    model = Model(inputs=X_indices, outputs=dense_2) 

    return model

In [92]:
def biDirect_gruArch(input_shape):

    X_indices = Input(input_shape)

    embeddings = embedding_layer(X_indices) 

    gruLayer = Bidirectional(GRU(128,activation='relu'))(embeddings) 

    dropoutLayer = Dropout(0.1)(gruLayer) 
    
    dense_1 = Dense(128, activation='relu')(dropoutLayer)

    dense_2 = Dense(3, activation='softmax')(dense_1)

    model = Model(inputs=X_indices, outputs=dense_2) 

    return model

**Training the Bidirectional LSTM model**

In [93]:
lstmBiDirect_model = biDirect_lstmArch((maxLen,))
lstmBiDirect_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 100)          469300    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 387       
Total params: 737,079
Trainable params: 737,079
Non-trainable params: 0
_________________________________________________________________


In [94]:
adam = keras.optimizers.Adam(lr = 0.00001) # 0.0001, 0.0002, 0.00001
lstmBiDirect_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
lstmBiDirect_model.fit(X_train_indices, y_train_labels, batch_size=4, epochs=100)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
40/40 [==============================] - 8s 200ms/step - loss: 1.1459 - acc: 0.2750
Epoch 2/100
40/40 [==============================] - 4s 88ms/step - loss: 1.1490 - acc: 0.2250
Epoch 3/100
40/40 [==============================] - 3s 87ms/step - loss: 1.1359 - acc: 0.2750
Epoch 4/100
40/40 [==============================] - 3s 86ms/step - loss: 1.1259 - acc: 0.3500
Epoch 5/100
40/40 [==============================] - 3s 87ms/step - loss: 1.1226 - acc: 0.2500
Epoch 6/100
40/40 [==============================] - 4s 88ms/step - loss: 1.1127 - acc: 0.2500
Epoch 7/100
40/40 [==============================] - 4s 90ms/step - loss: 1.1094 - acc: 0.2750
Epoch 8/100
40/40 [==============================] - 4s 89ms/step - loss: 1.1048 - acc: 0.3000
Epoch 9/100
40/40 [==============================] - 4s 88ms/step - loss: 1.1007 - acc: 0.3250
Epoch 10/100
40/40 [==============================] - 4s 90ms/step - loss: 1.0943 - acc: 0.3250
Epoch 11/100
40/40 [============================

40/40 [==============================] - 4s 100ms/step - loss: 0.8868 - acc: 0.5500
Epoch 87/100
40/40 [==============================] - 4s 98ms/step - loss: 0.8977 - acc: 0.5500
Epoch 88/100
40/40 [==============================] - 4s 101ms/step - loss: 0.8809 - acc: 0.5500
Epoch 89/100
40/40 [==============================] - 4s 110ms/step - loss: 0.8692 - acc: 0.5500
Epoch 90/100
40/40 [==============================] - 4s 108ms/step - loss: 0.8723 - acc: 0.5500
Epoch 91/100
40/40 [==============================] - 4s 112ms/step - loss: 0.8619 - acc: 0.5750
Epoch 92/100
40/40 [==============================] - 4s 110ms/step - loss: 0.8569 - acc: 0.5750
Epoch 93/100
40/40 [==============================] - 4s 108ms/step - loss: 0.8705 - acc: 0.5500
Epoch 94/100
40/40 [==============================] - 4s 93ms/step - loss: 0.8697 - acc: 0.5750
Epoch 95/100
40/40 [==============================] - 4s 92ms/step - loss: 0.8724 - acc: 0.5500
Epoch 96/100
40/40 [==========================

In [95]:
lstmBiDirect_model.evaluate(X_test_indices, y_test_labels)

10/10 [==============================] - 1s 117ms/step


[1.0449336767196655, 0.4000000059604645]

In [96]:
preds3 = lstmBiDirect_model.predict(X_test_indices)

In [97]:
print('predicted: ',preds3)
print('actual: ',y_test_labels)

predicted:  [[0.4511565  0.2950424  0.25380117]
 [0.42751592 0.2919193  0.28056479]
 [0.45007935 0.30925187 0.24066877]
 [0.5135472  0.30072927 0.18572363]
 [0.56941104 0.2521069  0.1784821 ]
 [0.56104904 0.2867718  0.15217914]
 [0.57387483 0.25228184 0.17384334]
 [0.3839347  0.31038865 0.3056767 ]
 [0.52486134 0.27523154 0.19990712]
 [0.5147425  0.28667885 0.19857861]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [98]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_6 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(preds3,axis=1), target_names=target_names)
print(clsf_rep_6)

                        precision    recall  f1-score   support

      Expect inflation       0.40      1.00      0.57         4
Inflation will go away       0.00      0.00      0.00         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.40        10
             macro avg       0.13      0.33      0.19        10
          weighted avg       0.16      0.40      0.23        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [99]:
n = np.random.randint(0,10)
print(n)
print(X_test[n])
print(y_test[n])
print(y_test_encoded[n])
print('acual: ',y_test_labels[n])
print('predicted: ',preds3[n])

6
the personal consumption expenditures index showed prices increased 55 percent last month as consumer spending pulled back  the federal reserves preferred inflation measure is showing signs of moderating after months of rapid price increases and a closely watched gauge of consumer spending slowed last month a sign that the economy may have less steam as it heads into 2023  the personal consumption expenditures price index climbed 55 percent in november from a year earlier a slowdown from 61 percent in the previous reading stripped of food and fuel costs which jump around a socalled core price measure climbed 47 percent down from 5 percent in the previous reading both figures were roughly in line with economist forecasts  although inflation is slowing it still has a long way to go to return to a more normal pace the fed has raised interest rates at the fastest clip in decades this year as it has tried to temper consumer and business demand hoping to force price increases to moderate t

**Training the Bidirectional GRU model**

In [120]:
gruBiDirect_model = biDirect_gruArch((maxLen,))
gruBiDirect_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 150)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 150, 100)          469300    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               175872    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_19 (Dense)             (None, 3)                 387       
Total params: 678,455
Trainable params: 678,455
Non-trainable params: 0
_________________________________________________________________


In [121]:
adam = keras.optimizers.Adam(lr = 0.00001) # 0.0001, 0.0002, 0.00001
gruBiDirect_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
gruBiDirect_model.fit(X_train_indices, y_train_labels, batch_size=4, epochs=100)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
40/40 [==============================] - 6s 162ms/step - loss: 1.0084 - acc: 0.5500
Epoch 2/100
40/40 [==============================] - 2s 60ms/step - loss: 1.0037 - acc: 0.5500
Epoch 3/100
40/40 [==============================] - 2s 54ms/step - loss: 1.0113 - acc: 0.5500
Epoch 4/100
40/40 [==============================] - 3s 81ms/step - loss: 0.9979 - acc: 0.5500
Epoch 5/100
40/40 [==============================] - 4s 88ms/step - loss: 0.9907 - acc: 0.5500
Epoch 6/100
40/40 [==============================] - 2s 60ms/step - loss: 0.9941 - acc: 0.5500
Epoch 7/100
40/40 [==============================] - 2s 54ms/step - loss: 0.9917 - acc: 0.5500
Epoch 8/100
40/40 [==============================] - 2s 54ms/step - loss: 0.9929 - acc: 0.5500
Epoch 9/100
40/40 [==============================] - 2s 61ms/step - loss: 0.9764 - acc: 0.5500
Epoch 10/100
40/40 [==============================] - 3s 67ms/step - loss: 1.0083 - acc: 0.5500
Epoch 11/100
40/40 [============================

40/40 [==============================] - 4s 110ms/step - loss: 0.8154 - acc: 0.5750
Epoch 87/100
40/40 [==============================] - 4s 102ms/step - loss: 0.8181 - acc: 0.5500
Epoch 88/100
40/40 [==============================] - 4s 90ms/step - loss: 0.8199 - acc: 0.5500
Epoch 89/100
40/40 [==============================] - 4s 100ms/step - loss: 0.8185 - acc: 0.5500
Epoch 90/100
40/40 [==============================] - 4s 98ms/step - loss: 0.8110 - acc: 0.5750
Epoch 91/100
40/40 [==============================] - 3s 75ms/step - loss: 0.8099 - acc: 0.5500
Epoch 92/100
40/40 [==============================] - 3s 78ms/step - loss: 0.7933 - acc: 0.5750
Epoch 93/100
40/40 [==============================] - 3s 82ms/step - loss: 0.8084 - acc: 0.5500
Epoch 94/100
40/40 [==============================] - 3s 71ms/step - loss: 0.8178 - acc: 0.5500
Epoch 95/100
40/40 [==============================] - 3s 68ms/step - loss: 0.7685 - acc: 0.5500
Epoch 96/100
40/40 [==============================

In [122]:
gruBiDirect_model.evaluate(X_test_indices, y_test_labels)

10/10 [==============================] - 1s 114ms/step


[1.0506579875946045, 0.4000000059604645]

In [123]:
preds4 = lstmBiDirect_model.predict(X_test_indices)

In [124]:
print('predicted: ',preds4)
print('actual: ',y_test_labels)

predicted:  [[0.4520835  0.29458362 0.25333294]
 [0.42846352 0.2913427  0.28019378]
 [0.45104796 0.30877382 0.24017815]
 [0.5135615  0.3007609  0.18567765]
 [0.5695871  0.25201064 0.17840219]
 [0.5610565  0.2867719  0.15217155]
 [0.5739395  0.2521159  0.17394456]
 [0.3847256  0.31008738 0.30518705]
 [0.52506983 0.27520606 0.19972406]
 [0.51486677 0.2865788  0.19855438]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [125]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_7 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(preds4,axis=1), target_names=target_names)
print(clsf_rep_7)

                        precision    recall  f1-score   support

      Expect inflation       0.40      1.00      0.57         4
Inflation will go away       0.00      0.00      0.00         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.40        10
             macro avg       0.13      0.33      0.19        10
          weighted avg       0.16      0.40      0.23        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [126]:
n = np.random.randint(0,10)
print(n)
print(X_test[n])
print(y_test[n])
print(y_test_encoded[n])
print('acual: ',y_test_labels[n])
print('predicted: ',preds4[n])

3
economic data on friday brought troubling news for federal reserve officials who are trying to rein in the fastest inflation in decades prices are still rising quickly wages are rising rapidly too and the strong consumer demand that is helping to fuel the inflationary fire shows little sign of letting up  the data from two separate government reports wasnt a surprise and included hints of progress but it was confirmation of the challenges facing policymakers and further evidence that their aggressive efforts to constrain the economy are taking time to have a significant effect  the feds preferred measure of inflation the personal consumption expenditures price index climbed 62 percent in the year through september in line with the increase the month before the commerce department said friday after stripping out food and fuel which can be volatile from month to month prices increased 51 percent over the past year a brisker increase than the 49 percent in the year through august  both 

**Revisiting GRU**

In [127]:
# Adjusting max len to 300
maxLenG2 = 300 ## 150 or 300

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, 
                            input_length=maxLenG2, weights = [emb_matrix], trainable=True)


X_train_indicesG2 = tokenizer.texts_to_sequences(X_train_prepros)
X_train_indicesG2 = pad_sequences(X_train_indices, maxlen=maxLenG2, padding='post')

X_test_indicesG2 = tokenizer.texts_to_sequences(X_test_prepros)
X_test_indicesG2 = pad_sequences(X_test_indices, maxlen=maxLenG2, padding='post')

In [128]:
gruGloVe_modelG2 = gruArchitecture((maxLenG2,))
gruGloVe_modelG2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 300)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 300, 100)          469300    
_________________________________________________________________
gru_5 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_21 (Dense)             (None, 3)                 387       
Total params: 574,135
Trainable params: 574,135
Non-trainable params: 0
_________________________________________________________________


In [129]:
# Starting with smallest lr from LSTM. If in doubt I will test 0.00005
adamG2 = keras.optimizers.Adam(lr = 0.00001) # 0.0001, 0.0002, 0.00001
gruGloVe_modelG2.compile(optimizer=adamG2, loss='categorical_crossentropy', metrics=['accuracy'])
gruGloVe_modelG2.fit(X_train_indicesG2, y_train_labels, batch_size=4, epochs=100)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
40/40 [==============================] - 9s 224ms/step - loss: 1.0986 - acc: 0.5500
Epoch 2/100
40/40 [==============================] - 6s 144ms/step - loss: 1.0984 - acc: 0.5500
Epoch 3/100
40/40 [==============================] - 6s 140ms/step - loss: 1.0982 - acc: 0.5500
Epoch 4/100
40/40 [==============================] - 5s 129ms/step - loss: 1.0980 - acc: 0.5500
Epoch 5/100
40/40 [==============================] - 5s 135ms/step - loss: 1.0978 - acc: 0.5500
Epoch 6/100
40/40 [==============================] - 5s 127ms/step - loss: 1.0976 - acc: 0.5500
Epoch 7/100
40/40 [==============================] - 5s 128ms/step - loss: 1.0974 - acc: 0.5500
Epoch 8/100
40/40 [==============================] - 5s 130ms/step - loss: 1.0972 - acc: 0.5500
Epoch 9/100
40/40 [==============================] - 5s 130ms/step - loss: 1.0970 - acc: 0.5500
Epoch 10/100
40/40 [==============================] - 5s 137ms/step - loss: 1.0967 - acc: 0.5500
Epoch 11/100
40/40 [===================

40/40 [==============================] - 5s 123ms/step - loss: 1.0666 - acc: 0.5500
Epoch 86/100
40/40 [==============================] - 5s 115ms/step - loss: 1.0636 - acc: 0.5500
Epoch 87/100
40/40 [==============================] - 4s 110ms/step - loss: 1.0626 - acc: 0.5500
Epoch 88/100
40/40 [==============================] - 5s 119ms/step - loss: 1.0609 - acc: 0.5500
Epoch 89/100
40/40 [==============================] - 5s 122ms/step - loss: 1.0614 - acc: 0.5500
Epoch 90/100
40/40 [==============================] - 5s 116ms/step - loss: 1.0624 - acc: 0.5500
Epoch 91/100
40/40 [==============================] - 5s 130ms/step - loss: 1.0589 - acc: 0.5500
Epoch 92/100
40/40 [==============================] - 5s 135ms/step - loss: 1.0589 - acc: 0.5500
Epoch 93/100
40/40 [==============================] - 5s 134ms/step - loss: 1.0587 - acc: 0.5500
Epoch 94/100
40/40 [==============================] - 5s 126ms/step - loss: 1.0580 - acc: 0.5500
Epoch 95/100
40/40 [=======================

In [130]:
gruGloVe_modelG2.evaluate(X_test_indicesG2, y_test_labels)

10/10 [==============================] - 1s 81ms/step


[1.0799920558929443, 0.4000000059604645]

In [131]:
predsG2 = gruGloVe_modelG2.predict(X_test_indicesG2)
print('predicted: ',predsG2)
print('actual: ',y_test_labels)

predicted:  [[0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]
 [0.3881607  0.31308183 0.29875743]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [132]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_8 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(predsG2,axis=1), target_names=target_names)
print(clsf_rep_8)

                        precision    recall  f1-score   support

      Expect inflation       0.40      1.00      0.57         4
Inflation will go away       0.00      0.00      0.00         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.40        10
             macro avg       0.13      0.33      0.19        10
          weighted avg       0.16      0.40      0.23        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

**Another try with new parameters GRU architecture**

In [133]:
# Adjusting max len to 300
maxLenG3 = 150 ## 150 or 300

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, 
                            input_length=maxLenG3, weights = [emb_matrix], trainable=True)


X_train_indicesG3 = tokenizer.texts_to_sequences(X_train_prepros)
X_train_indicesG3 = pad_sequences(X_train_indices, maxlen=maxLenG3, padding='post')

X_test_indicesG3 = tokenizer.texts_to_sequences(X_test_prepros)
X_test_indicesG3 = pad_sequences(X_test_indices, maxlen=maxLenG3, padding='post')

In [134]:
gruGloVe_modelG3 = gruArchitecture((maxLenG3,))
gruGloVe_modelG3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 150)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 150, 100)          469300    
_________________________________________________________________
gru_6 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_23 (Dense)             (None, 3)                 387       
Total params: 574,135
Trainable params: 574,135
Non-trainable params: 0
_________________________________________________________________


In [135]:
# Starting with smallest lr from LSTM. If in doubt I will test 0.00005
adamG3 = keras.optimizers.Adam(lr = 0.000001) # 0.0001, 0.0002, 0.00001
gruGloVe_modelG3.compile(optimizer=adamG3, loss='categorical_crossentropy', metrics=['accuracy'])
gruGloVe_modelG3.fit(X_train_indicesG3, y_train_labels, batch_size=4, epochs=150)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/150
40/40 [==============================] - 5s 122ms/step - loss: 1.0786 - acc: 0.2750
Epoch 2/150
40/40 [==============================] - 3s 63ms/step - loss: 1.0871 - acc: 0.2750
Epoch 3/150
40/40 [==============================] - 2s 61ms/step - loss: 1.0824 - acc: 0.2500
Epoch 4/150
40/40 [==============================] - 2s 60ms/step - loss: 1.0787 - acc: 0.3250
Epoch 5/150
40/40 [==============================] - 3s 67ms/step - loss: 1.0734 - acc: 0.3500
Epoch 6/150
40/40 [==============================] - 3s 64ms/step - loss: 1.0628 - acc: 0.3500
Epoch 7/150
40/40 [==============================] - 3s 64ms/step - loss: 1.0639 - acc: 0.3750
Epoch 8/150
40/40 [==============================] - 3s 63ms/step - loss: 1.0571 - acc: 0.3250
Epoch 9/150
40/40 [==============================] - 3s 74ms/step - loss: 1.0765 - acc: 0.3000
Epoch 10/150
40/40 [==============================] - 3s 73ms/step - loss: 1.0821 - acc: 0.2750
Epoch 11/150
40/40 [============================

40/40 [==============================] - 2s 61ms/step - loss: 1.0498 - acc: 0.4750
Epoch 87/150
40/40 [==============================] - 2s 61ms/step - loss: 1.0482 - acc: 0.4000
Epoch 88/150
40/40 [==============================] - 2s 59ms/step - loss: 1.0582 - acc: 0.3750
Epoch 89/150
40/40 [==============================] - 2s 60ms/step - loss: 1.0620 - acc: 0.3750
Epoch 90/150
40/40 [==============================] - 3s 65ms/step - loss: 1.0629 - acc: 0.4250
Epoch 91/150
40/40 [==============================] - 3s 66ms/step - loss: 1.0609 - acc: 0.3750
Epoch 92/150
40/40 [==============================] - 3s 64ms/step - loss: 1.0532 - acc: 0.4250
Epoch 93/150
40/40 [==============================] - 2s 60ms/step - loss: 1.0527 - acc: 0.4000
Epoch 94/150
40/40 [==============================] - 3s 66ms/step - loss: 1.0490 - acc: 0.4000
Epoch 95/150
40/40 [==============================] - 3s 77ms/step - loss: 1.0670 - acc: 0.3750
Epoch 96/150
40/40 [==============================] -

In [136]:
gruGloVe_modelG3.evaluate(X_test_indicesG3, y_test_labels)

10/10 [==============================] - 1s 120ms/step


[0.9966440200805664, 0.5]

In [137]:
predsG3 = gruGloVe_modelG3.predict(X_test_indicesG3)
print('predicted: ',predsG3)
print('actual: ',y_test_labels)

predicted:  [[0.3360129  0.33206347 0.33192357]
 [0.33601475 0.33206233 0.33192298]
 [0.3360129  0.33206347 0.33192357]
 [0.38113654 0.395516   0.22334747]
 [0.4143002  0.35358083 0.23211892]
 [0.40796062 0.38618457 0.20585483]
 [0.36244792 0.39137244 0.2461796 ]
 [0.3360129  0.33206347 0.33192357]
 [0.3634844  0.4212177  0.21529791]
 [0.43033394 0.37529197 0.1943741 ]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [138]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_9 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(predsG3,axis=1), target_names=target_names)
print(clsf_rep_9)

                        precision    recall  f1-score   support

      Expect inflation       0.43      0.75      0.55         4
Inflation will go away       0.67      0.40      0.50         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.50        10
             macro avg       0.37      0.38      0.35        10
          weighted avg       0.50      0.50      0.47        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [139]:
print('LSTM')
print('MaxLen=300 | lr=0.0001 | epochs=100')
print(clsf_rep_1)
print('MaxLen=300 | lr=0.00001 | epochs=100')
print(clsf_rep_2)
print('MaxLen=300 | lr=0.0001 | epochs=150')
print(clsf_rep_3)
print('MaxLen=150 | lr=0.00005 | epochs=100')
print(clsf_rep_4)

print('GRU')
print('MaxLen=150 | lr=0.00001 | epochs=100')
print(clsf_rep_5)
print('MaxLen=300 | lr=0.00001 | epochs=100')
print(clsf_rep_8)
print('MaxLen=150 | lr=0.00001 | epochs=150')
print(clsf_rep_9)

print('Bidirectional LSTM')
print('MaxLen=150 | lr=0.00001 | epochs=100')
print(clsf_rep_6)

print('Bidirectional GRU')
print('MaxLen=150 | lr=0.00001 | epochs=100')
print(clsf_rep_7)

LSTM
MaxLen=300 | lr=0.0001 | epochs=100
                        precision    recall  f1-score   support

      Expect inflation       0.67      0.50      0.57         4
Inflation will go away       0.71      1.00      0.83         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.70        10
             macro avg       0.46      0.50      0.47        10
          weighted avg       0.62      0.70      0.65        10

MaxLen=300 | lr=0.00001 | epochs=100
                        precision    recall  f1-score   support

      Expect inflation       0.50      1.00      0.67         4
Inflation will go away       1.00      0.40      0.57         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.60        10
             macro avg       0.50      0.47      0.41        10
          weighted avg       0.70      0.60      0.55        10

MaxLen=300 | lr=0.00

### <--------- Project Ends Here --------->

In [141]:
# Adjusting max len to 300
maxLenG4 = 500 ## 150 or 300

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, 
                            input_length=maxLenG4, weights = [emb_matrix], trainable=True)


X_train_indicesG4 = tokenizer.texts_to_sequences(X_train_prepros)
X_train_indicesG4 = pad_sequences(X_train_indices, maxlen=maxLenG4, padding='post')

X_test_indicesG4 = tokenizer.texts_to_sequences(X_test_prepros)
X_test_indicesG4 = pad_sequences(X_test_indices, maxlen=maxLenG4, padding='post')

In [142]:
gruGloVe_modelG4 = gruArchitecture((maxLenG4,))
gruGloVe_modelG4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 500)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 500, 100)          469300    
_________________________________________________________________
gru_7 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 387       
Total params: 574,135
Trainable params: 574,135
Non-trainable params: 0
_________________________________________________________________


In [143]:
# Starting with smallest lr from LSTM. If in doubt I will test 0.00005
adamG4 = keras.optimizers.Adam(lr = 0.000001) # 0.0001, 0.0002, 0.00001
gruGloVe_modelG4.compile(optimizer=adamG4, loss='categorical_crossentropy', metrics=['accuracy'])
gruGloVe_modelG4.fit(X_train_indicesG4, y_train_labels, batch_size=4, epochs=150)

C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/150
40/40 [==============================] - 9s 214ms/step - loss: 1.0986 - acc: 0.3250
Epoch 2/150
40/40 [==============================] - 7s 169ms/step - loss: 1.0986 - acc: 0.5500
Epoch 3/150
40/40 [==============================] - 11s 266ms/step - loss: 1.0986 - acc: 0.5500
Epoch 4/150
40/40 [==============================] - 10s 242ms/step - loss: 1.0986 - acc: 0.5500
Epoch 5/150
40/40 [==============================] - 9s 234ms/step - loss: 1.0986 - acc: 0.5500
Epoch 6/150
40/40 [==============================] - 9s 235ms/step - loss: 1.0986 - acc: 0.5500
Epoch 7/150
40/40 [==============================] - 10s 239ms/step - loss: 1.0986 - acc: 0.5500
Epoch 8/150
40/40 [==============================] - 10s 248ms/step - loss: 1.0986 - acc: 0.5500
Epoch 9/150
40/40 [==============================] - 10s 245ms/step - loss: 1.0986 - acc: 0.5500
Epoch 10/150
40/40 [==============================] - 10s 242ms/step - loss: 1.0986 - acc: 0.5500
Epoch 11/150
40/40 [=============

40/40 [==============================] - 9s 225ms/step - loss: 1.0984 - acc: 0.5500
Epoch 86/150
40/40 [==============================] - 10s 244ms/step - loss: 1.0984 - acc: 0.5500
Epoch 87/150
40/40 [==============================] - 9s 233ms/step - loss: 1.0984 - acc: 0.5500
Epoch 88/150
40/40 [==============================] - 10s 243ms/step - loss: 1.0984 - acc: 0.5500
Epoch 89/150
40/40 [==============================] - 9s 237ms/step - loss: 1.0984 - acc: 0.5500
Epoch 90/150
40/40 [==============================] - 9s 224ms/step - loss: 1.0984 - acc: 0.5500
Epoch 91/150
40/40 [==============================] - 10s 261ms/step - loss: 1.0984 - acc: 0.5500
Epoch 92/150
40/40 [==============================] - 11s 274ms/step - loss: 1.0984 - acc: 0.5500
Epoch 93/150
40/40 [==============================] - 11s 272ms/step - loss: 1.0984 - acc: 0.5500
Epoch 94/150
40/40 [==============================] - 10s 258ms/step - loss: 1.0984 - acc: 0.5500
Epoch 95/150
40/40 [=================

In [144]:
gruGloVe_modelG4.evaluate(X_test_indicesG4, y_test_labels)

10/10 [==============================] - 2s 153ms/step


[1.098435878753662, 0.4000000059604645]

In [145]:
predsG4 = gruGloVe_modelG4.predict(X_test_indicesG4)
print('predicted: ',predsG4)
print('actual: ',y_test_labels)

predicted:  [[0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]
 [0.33370975 0.3331983  0.33309197]]
actual:  [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [146]:
target_names = ['Expect inflation', 'Inflation will go away', 'Neutral']
clsf_rep_10 = classification_report(np.argmax(y_test_labels,axis=1),np.argmax(predsG4,axis=1), target_names=target_names)
print(clsf_rep_10)

                        precision    recall  f1-score   support

      Expect inflation       0.40      1.00      0.57         4
Inflation will go away       0.00      0.00      0.00         5
               Neutral       0.00      0.00      0.00         1

              accuracy                           0.40        10
             macro avg       0.13      0.33      0.19        10
          weighted avg       0.16      0.40      0.23        10



C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rzamb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

**Bidirectional LSTM**

Follows Keras examples: https://keras.io/examples/nlp/bidirectional_lstm_imdb/

In [61]:
max_features = 20000  # Only consider the top 20k words
maxlen = 3000  # Only consider the first 3000 words of each article

In [67]:
 # Input for variable-length sequences of integers: shape=None
def biDirect_lstmArch(input_shape=(None,)): 

    inputs = Input(input_shape, dtype="int32")

    embeddings = embedding_layer(inputs)
    
    # Add 2 bidirectional LSTMs
    x = Bidirectional(LSTM(64, return_sequences=True))(embeddings)
    x = Bidirectional(LSTM(64))(x)
    
    # Add a classifier
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)

    #X = Dropout(0.6)(X)

    return model

In [68]:
biDict_lstmGloVe_model = biDirect_lstmArch()

In [69]:
biDict_lstmGloVe_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding (Embedding)        multiple                  432200    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 128)         84480     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 615,625
Trainable params: 183,425
Non-trainable params: 432,200
_________________________________________________________________


In [71]:
biDict_lstmGloVe_model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
biDict_lstmGloVe_model.fit(X_train_indices, y_train_encoded, batch_size=32, epochs=3)

Epoch 1/3
40/40 [==============================] - 353s 9s/step - loss: 0.7408 - acc: 0.5250
Epoch 2/3
40/40 [==============================] - 291s 7s/step - loss: 0.6709 - acc: 0.5750
Epoch 3/3
40/40 [==============================] - 341s 9s/step - loss: 0.6292 - acc: 0.2500


In [72]:
biDict_lstmGloVe_model.evaluate(X_test_indices, y_test_encoded)

10/10 [==============================] - 2s 206ms/step


[0.7221247553825378, 0.5]

In [73]:
preds2 = lstmGloVe_model.predict(X_test_indices)

In [74]:
print('predicted: ',preds2)
print('actual: ',y_test_encoded)

predicted:  [[0.50194865]
 [0.50194865]
 [0.50194865]
 [0.50194865]
 [0.50194865]
 [0.50194865]
 [0.50194865]
 [0.50194865]
 [0.50194865]
 [0.50194865]]
actual:  [1 0 1 0 0 1 0 0 1 1]


In [75]:
n = np.random.randint(0,10)
print(n)
print(X_test[n])
print(y_test[n])
print(y_test_encoded[n])

5
inflation used to be the scourge of the world economy and the bane of american presidents in 1971 amid an overheating economy richard nixon took to television to announce a freeze on “all prices and wages throughout the united states” a board of bureaucrats ruled on what this meant for everything from golf club memberships to commodity futures gerald ford nixon’s successor preferred a grassroots approach he distributed buttons bearing his slogan win for “whip inflation now” ronald reagan running for office four years later amid another surge in prices declared inflation to be “as violent as a mugger as frightening as an armed robber and as deadly as a hit man”  today the lethal assassin has gone missing most economies no longer struggle with runaway prices instead they find inflation is too low as judged by their inflation targets a decade of interest rates at or near rockbottom has not changed that nor has the printing of money by central banks in america the euro zone britain and j

In [76]:
sampleBody = X_test[n]
for i in range(len(articlesSample)):
    if articlesSample[i][2] == sampleBody:
        print(articlesSample[i])

labeledArticle(articleID=4, label='Inflation will fade away', body='inflation used to be the scourge of the world economy and the bane of american presidents in 1971 amid an overheating economy richard nixon took to television to announce a freeze on “all prices and wages throughout the united states” a board of bureaucrats ruled on what this meant for everything from golf club memberships to commodity futures gerald ford nixon’s successor preferred a grassroots approach he distributed buttons bearing his slogan win for “whip inflation now” ronald reagan running for office four years later amid another surge in prices declared inflation to be “as violent as a mugger as frightening as an armed robber and as deadly as a hit man”  today the lethal assassin has gone missing most economies no longer struggle with runaway prices instead they find inflation is too low as judged by their inflation targets a decade of interest rates at or near rockbottom has not changed that nor has the printin